things to try:
lemma all text
remove stopwords

In [8]:
#imports
import spacy
from spacy.util import minibatch
from spacy.training.example import Example
from spacy.matcher import PhraseMatcher
import numpy as np
import seaborn as sns
import pandas as pd
import random
from numpy import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')


In [9]:
df_train = pd.read_csv('train.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [5]:
print('location Nan: ',df_train.location.isna().sum())
print('keyword Nan: ',df_train.keyword.isna().sum())
print(df_train.info())

location Nan:  2533
keyword Nan:  61
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None


In [6]:
df_train.location.value_counts()[20:40]

United Kingdom       14
San Francisco        14
Los Angeles          13
Indonesia            13
Washington, D.C.     13
NYC                  12
Toronto              12
Ireland              12
San Francisco, CA    11
Seattle              11
Earth                11
Chicago              11
London, England      10
Texas                10
ss                   10
New York City        10
Atlanta, GA          10
London, UK           10
Sacramento, CA       10
Nashville, TN         9
Name: location, dtype: int64

### There are many fake places such as 'ss' or 'Worldwide' so I think we can just drop the whole column for this dataset

In [7]:
df_train.drop(columns = ['location'],inplace=True)

In [8]:
df_train.keyword.value_counts().head(10)

fatalities     45
armageddon     42
deluge         42
damage         41
sinking        41
body%20bags    41
harm           41
windstorm      40
fear           40
evacuate       40
Name: keyword, dtype: int64

## This column seems useful so we will keep it, drop the nan's since its only 61 rows and replace %20 with spaces

In [9]:
df_train.keyword = df_train.keyword.str.replace('%20',' ')

In [10]:
df_train.keyword.value_counts().head(10)

fatalities    45
deluge        42
armageddon    42
body bags     41
sinking       41
harm          41
damage        41
twister       40
windstorm     40
evacuate      40
Name: keyword, dtype: int64

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       7613 non-null   int64 
 1   keyword  7552 non-null   object
 2   text     7613 non-null   object
 3   target   7613 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 238.0+ KB


In [12]:
df_train.dropna(inplace=True)

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7552 entries, 31 to 7582
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       7552 non-null   int64 
 1   keyword  7552 non-null   object
 2   text     7552 non-null   object
 3   target   7552 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 295.0+ KB


## Lets add the keyword to as the custom extension for the text and then train the model

In [14]:
from spacy.lang.en import English
from spacy.tokens import Doc, Span

def get_keyword(doc):
    # Return if any of the tokens in the doc return True for token.like_num
    return df_train.keyword[df_train.index[df_train.text == doc.text]]

# Register the Doc property extension "has_number" with the getter get_has_number
Doc.set_extension("keyword", getter=get_keyword)

## We can try to process it with links then we can try without the links

In [12]:
def split_data(data,which_data, split=0.8):
    
    # Shuffle data
    train_data = data.sample(frac=1, random_state=7)
    
    texts = data[which_data]
    labels = [{"POSITIVE": bool(y), "NEGATIVE": not bool(y)}
              for y in train_data.target]
    split = int(len(train_data) * split)
    
    train_labels = [{"cats": labels} for labels in labels[:split]]
    val_labels = [{"cats": labels} for labels in labels[split:]]
    
    return texts[:split], train_labels, texts[split:], val_labels



In [17]:
import random
import spacy


from spacy.util import minibatch
from spacy.training.example import Example


def train_data(nlp, text,labels):
    textcat = nlp.add_pipe("textcat")
    # Add labels to text classifier
    textcat.add_label("POSITIVE")
    textcat.add_label("NEGATIVE")
    random.seed(1)
    spacy.util.fix_random_seed(1)
    optimizer = nlp.begin_training()
    losses = {}
    train_data = list(zip(train_texts, train_labels))
    for epoch in range(10):
        # Create the batch generator with batch size = 8
        batches = minibatch(train_data, size=8)
        # Iterate through minibatches
        for batch in batches:
            for text, labels in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, labels)
                nlp.update([example], sgd=optimizer, losses=losses)
        print(losses)
    

In [9]:
def predict(nlp, texts): 
    # Use the model's tokenizer to tokenize each input text
    docs = [nlp.tokenizer(text) for text in texts]
    
    # Use textcat to get the scores for each doc
    textcat = nlp.get_pipe('textcat')
    scores = textcat.predict(docs)
    # From the scores, find the class with the highest score/probability
    predicted_class = scores.argmax(axis=1)
    
    return predicted_class


In [10]:

def evaluate(model, texts, labels):
    """ Returns the accuracy of a TextCategorizer model. 
    
        Arguments
        ---------
        model: ScaPy model with a TextCategorizer
        texts: Text samples, from load_data function
        labels: True labels, from load_data function
    
    """
    # Get predictions from textcat model (using your predict method)
    predicted_class = predict(model,texts)
    
    # From labels, get the true class as a list of integers (POSITIVE -> 1, NEGATIVE -> 0)
    true_class = [int(each['cats']['POSITIVE']) for each in labels]
    
    # A boolean or int array indicating correct predictions
    correct_predictions = predicted_class == true_class
    
    # The accuracy, number of correct predictions divided by all predictions
    accuracy = correct_predictions.mean()
    
    return accuracy


In [18]:
print(evaluate(nlp,val_texts,val_labels))

0.29649238914626075


Well, thats terrible.. I guess it's time to cut the link and do lemmatization

In [15]:
import re
df_train['text'] = df_train['text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
df_train.head(20)

,id,keyword,text,target
31,48,ablaze,@bbcmtd Wholesale Markets ablaze,1
32,49,ablaze,We always try to bring the heavy. #metal #RT,0
33,50,ablaze,#AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba.,1
34,52,ablaze,Crying out for more! Set me ablaze,0
35,53,ablaze,On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE,0
36,54,ablaze,@PhDSquares #mufc they've built so much hype around new acquisitions but I doubt they will set the EPL ablaze this season.,0
37,55,ablaze,INEC Office in Abia Set Ablaze -,1
38,56,ablaze,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...,1
39,57,ablaze,Ablaze for you Lord :D,0
40,59,ablaze,Check these out:,0


The Hashtags might be useful so we will leave it and the 

In [41]:
import re

df_train['text'] = [ re.sub( "@[^\s]+", "",x) for x in df_train['text'] ]
df_train.text.head()

31                                       Wholesale Markets ablaze 
32                   We always try to bring the heavy. #metal #RT 
33    #AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. 
34                              Crying out for more! Set me ablaze
35          On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE 
Name: text, dtype: object

## time to lemma
### we will just add it to the pipeline and retrain/re-evaluate

In [43]:
nlp = spacy.blank('en')
config = {"mode": "rule", "overwrite": True}
textcat = nlp.add_pipe("textcat")
# make sure the pipeline is in the right order
print(nlp.pipe_names)

['textcat']


In [46]:
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")
random.seed(1)
spacy.util.fix_random_seed(1)

def train_data_revised(text,labels):
    optimizer = nlp.begin_training()
    losses = {}
    train_data = list(zip(train_texts, train_labels))
    for epoch in range(10):
        # Create the batch generator with batch size = 8
        batches = minibatch(train_data, size=8)
        # Iterate through minibatches
        for batch in batches:
            for text, labels in batch:
                doc = nlp.make_doc(text)
                doc = nlp.make_doc(' '.join([token.lemma_ for token in doc]))
                example = Example.from_dict(doc, labels)
                nlp.update([example], sgd=optimizer, losses=losses)
        print(losses)
        

training time

In [47]:
# resplit data
train_texts, train_labels,val_texts, val_labels = split_data(df_train)
train_data_revised(train_texts,train_labels)
print(evaluate(nlp,val_texts,val_labels))

{'textcat': 3408.785533385788}
{'textcat': 6571.3855210339825}
{'textcat': 9710.419881779904}
{'textcat': 12809.537989072705}
{'textcat': 15872.08745364772}
{'textcat': 18909.73925317239}
{'textcat': 21939.149305348023}
{'textcat': 24946.068432995788}
{'textcat': 27889.443326120934}
{'textcat': 30821.16531138934}
0.4116479152878888


big improvement! how can we improve it further?
- vectorization?
- remove stop words
- retrain with all data
- add keyword to sentence instead of an extension

In [1]:
train_texts[0]

NameError: name 'train_texts' is not defined

In [4]:
import re
import spacy

def clean_data(df_train):
    
    
    # remove locations
    df_train.drop(columns = ['location'],inplace=True)
    # fixing keywords from spaces
    df_train.keyword = df_train.keyword.str.replace('%20',' ')
    # removing rows with nan
    df_train.dropna(inplace=True)
    # remove links and twitter @ tags
    df_train['text'] = df_train['text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
    df_train['text'] = [ re.sub( "@[^\s]+", "",x) for x in df_train['text'] ]
    # removing the # from texts
    df_train['text'] = df_train['text'].replace('#','')
    # Remove all punctuation
    df_train['text'] = df_train['text'].str.replace('[^\w\s]',' ')
    # add keyword to text
    df_train['text'] = df_train['keyword']+' '+ df_train['text']
    # lemmatize text
    nlp = spacy.load('en_core_web_md')
    lemma_text = []
    vector_text = []
    for text in df_train['text']:
        lemmatized_sentence = []
        doc = nlp(text)
        vector_text.append(doc.vector)
        for word in doc:
            lemmatized_sentence.append(word.lemma_)
        lemma_text.append(' '.join(lemmatized_sentence))
    df_train['lemma_text'] = lemma_text
    df_train['vector_text'] = vector_text
    

In [4]:
clean_data(df_train)

In [19]:
df_train.head()

,id,keyword,text,target,lemma_text,vector_text
31,48,ablaze,ablaze Wholesale Markets ablaze,1,ablaze Wholesale Markets ablaze,"[-0.0563236, 0.257118, 0.1955008, 0.0688708, 0.182452, -0.155606, 0.113748, 0.35473198, -0.1426564, 1.160272, -0.07376599, -0.29792398, -0.099010006, -0.27831, -0.09935601, -0.11009719, 0.32681003, 0.807452, 0.22835799, -0.09993, -0.065302, 0.036398, 0.005114003, -0.042065807, 0.09796201, 0.14403799, -0.307568, 0.123176, -0.014593994, -0.015843997, -0.012040001, -0.07972039, -0.20578401, 0.09755139, -0.036065996, 0.0068355976, 0.18506669, 0.144444, 0.14343621, 0.0418124, 0.022623003, 0.0041340007, 0.24607602, -0.22774442, 0.05598221, 0.0065254, 0.056873996, -0.178454, -0.1993216, 0.161984, 0.063133396, 0.2606274, -0.028442722, 0.25649, 0.16508639, -0.12415598, -0.333006, 0.088394, -0.011596799, -0.19837399, -0.089459196, -0.16033998, -0.004598403, -0.030435601, -0.06268401, -0.1724724, 0.2821346, -0.09074, 0.106261395, -0.110604204, 0.1158576, -0.17723, -0.14292601, 0.214138, 0.145776, 0.0128617585, 0.1582818, 0.031759597, 0.025170002, -0.090425596, 0.048223794, 0.202898, -0.1456964, -0.062476397, 0.15210919, 0.15843621, 0.10579141, 0.59602606, -0.039108, 0.1184654, -0.04655268, -0.0051670047, 0.32464108, -0.191902, 0.0020141988, 0.0587912, -0.11395688, 0.18360801, -0.114694, -0.1219092, ...]"
32,49,ablaze,ablaze We always try to bring the heavy metal RT,0,ablaze we always try to bring the heavy metal RT,"[0.045223314, 0.1333939, -0.050343323, 0.08015067, 0.058838334, -0.0554654, -0.018545, 0.08634857, -0.051001832, 1.5571651, -0.10734125, -0.10247001, 0.13837694, 0.029538453, -0.079866245, -0.13390766, 0.044125587, 1.0139709, -0.1453875, 0.029887335, 0.09791225, -0.0023505837, -0.044185847, -0.060278002, -0.06502458, 0.016685084, -0.10645178, -0.074082986, 0.099368095, -0.07184333, 0.007993648, 0.0489028, -0.105595164, 0.121057004, 0.11047691, -0.023924187, 0.1304792, 0.09394637, 0.026132086, -0.04840246, -0.032212753, 0.023670083, -0.023923248, -0.19602084, 0.076127246, -0.04462072, 0.029789248, -0.10485867, -0.105620354, 0.07480307, 0.021529667, 0.101172, 0.09638619, 0.11223215, 0.029637339, 0.018065736, -0.16108476, -0.026385665, -0.051710915, -0.055401, -0.13327457, -0.04083883, -0.10042878, 0.14891459, 0.06408292, -0.06431223, 0.047274664, -0.042019755, 0.05255917, 0.059616495, -0.022005146, 0.0026573853, 0.110539995, 0.0010725869, 0.20260318, 0.01784075, 0.04255783, 0.009275745, -0.040025167, 0.0037872486, 0.018847583, 0.033973992, -0.23186707, -0.04028337, 0.10116502, -0.115347445, 0.037151914, 0.122359686, 0.1257649, 0.035692245, -0.072939165, -0.09851825, 0.08350283, 0.1036765, 0.012105833, 0.085228, 0.04201318, 0.017549252, -0.031674832, 0.004940662, ...]"
33,50,ablaze,ablaze AFRICANBAZE Breaking news Nigeria flag set ablaze in Aba,1,ablaze AFRICANBAZE break news Nigeria flag set ablaze in Aba,"[0.013710539, 0.14963509, 0.1158237, -0.021470385, -0.015969686, -0.08640769, -0.017800616, 0.11728731, -0.08619977, 1.017307, -0.15669984, -0.17209195, 0.0132249985, -0.039106157, -0.07251846, 0.004347696, 0.16670431, 0.5195946, 0.13820623, -0.13763155, -0.04797069, 0.014259151, 0.14222346, -0.041423623, 0.08688776, -0.04426492, -0.09938362, 0.13981031, 0.12833564, 0.0698008, 0.02618777, 0.0046466915, -0.07892693, 0.17477046, 0.059629228, 0.03305077, 0.022554385, 0.16193262, -0.033037618, -0.02115508, 0.10090612, 0.0062303073, 0.080808386, -0.07174353, -0.011538084, -0.10158007, 0.098384224, -0.04438454, -0.171915, -0.0044546905, -0.09982138, -0.027612928, -0.039756425, 0.029415388, -0.057097398, 0.009407693, 0.037795153, 0.053968847, 0.1611676, -0.06302346, -0.11900708, 0.120851554, -0.027273927, -0.030337691, 0.019895535, 0.014181694, 0.014484333, 0.0007102306, 0.12199794, 0.06954709, -0.03853, 0.010639996, 0.031774845, -0.08349548, 0.12122769, -0.0054409276, 0.029222231, -0.02278946, 0.051309228, 0.05586761, -0.05128553, 0.19871166, -0.034725077, 0.106526166, 0.12809578, 0.09827346, 0.17790054, 0.

## Now we can retrain with vectors and lemmatized text and see which performs better

### Lets do lemmatized text  followed by vectored text

In [20]:
nlp = spacy.blank('en')
train_texts, train_labels,val_texts, val_labels = split_data(df_train,'lemma_text')
train_data(nlp, train_texts, train_labels)
print(evaluate(nlp,val_texts,val_labels))



{'textcat': 3464.7708281795058}
{'textcat': 6647.314140336079}
{'textcat': 9738.455008731271}
{'textcat': 12530.436533370645}
{'textcat': 14761.559309078966}
{'textcat': 16487.391534169547}
{'textcat': 17848.317449445047}
{'textcat': 19085.826673614796}
{'textcat': 20206.240546356912}
{'textcat': 21265.321677953958}
0.48444738583719393


TypeError: Argument 'string' has incorrect type (expected str, got numpy.ndarray)

have to train vector text seperately

In [22]:
def train_vector_data(nlp, text,labels):
    textcat = nlp.add_pipe("textcat")
    # Add labels to text classifier
    textcat.add_label("POSITIVE")
    textcat.add_label("NEGATIVE")
    random.seed(1)
    spacy.util.fix_random_seed(1)
    optimizer = nlp.begin_training()
    losses = {}
    train_data = list(zip(train_texts, train_labels))
    for epoch in range(10):
        # Create the batch generator with batch size = 8
        batches = minibatch(train_data, size=8)
        # Iterate through minibatches
        for batch in batches:
            for text, labels in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc.vector, labels)
                nlp.update([example], sgd=optimizer, losses=losses)
        print(losses)
    

In [23]:
nlp = spacy.blank('en')
train_texts, train_labels,val_texts, val_labels = split_data(df_train,'lemma_text')
train_vector_data(nlp, train_texts, train_labels)
print(evaluate(nlp,val_texts,val_labels))


TypeError: Argument 'predicted' has incorrect type (expected spacy.tokens.doc.Doc, got numpy.ndarray)

In [24]:
train_texts.head()

31                                     ablaze   Wholesale Markets ablaze
32                 ablaze we always try to bring the heavy    metal   RT
33    ablaze   AFRICANBAZE   break news Nigeria flag set ablaze in Aba  
34                                ablaze cry out for more   set I ablaze
35           ablaze on plus side look at the sky last night IT be ABLAZE
Name: lemma_text, dtype: object

In [25]:
def split_vector_data(data, split=0.8):
    
    # Shuffle data
    train_data = data.sample(frac=1, random_state=7)
    
    texts = data['vector_text']
    labels = data['target']
    split = int(len(train_data) * split)
    
    return texts[:split], train_labels[:split], texts[split:], val_labels[split:]


In [27]:
train_texts, train_labels,val_texts, val_labels = split_vector_data(df_train)
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_texts, train_labels)
res_pred = clf.predict(val_texts)
score = accuracy_score(res_labels, res_pred)
print('accuracy: ',score)

ValueError: setting an array element with a sequence.

In [10]:
import numpy as np
import spacy
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
clean_data(df_train)
# Need to load the large model to get the vectors


In [11]:
df_train.head()

,id,keyword,text,target,lemma_text,vector_text
31,48,ablaze,ablaze Wholesale Markets ablaze,1,ablaze Wholesale Markets ablaze,"[-0.0563236, 0.257118, 0.1955008, 0.0688708, 0.182452, -0.155606, 0.113748, 0.35473198, -0.1426564, 1.160272, -0.07376599, -0.29792398, -0.099010006, -0.27831, -0.09935601, -0.11009719, 0.32681003, 0.807452, 0.22835799, -0.09993, -0.065302, 0.036398, 0.005114003, -0.042065807, 0.09796201, 0.14403799, -0.307568, 0.123176, -0.014593994, -0.015843997, -0.012040001, -0.07972039, -0.20578401, 0.09755139, -0.036065996, 0.0068355976, 0.18506669, 0.144444, 0.14343621, 0.0418124, 0.022623003, 0.0041340007, 0.24607602, -0.22774442, 0.05598221, 0.0065254, 0.056873996, -0.178454, -0.1993216, 0.161984, 0.063133396, 0.2606274, -0.028442722, 0.25649, 0.16508639, -0.12415598, -0.333006, 0.088394, -0.011596799, -0.19837399, -0.089459196, -0.16033998, -0.004598403, -0.030435601, -0.06268401, -0.1724724, 0.2821346, -0.09074, 0.106261395, -0.110604204, 0.1158576, -0.17723, -0.14292601, 0.214138, 0.145776, 0.0128617585, 0.1582818, 0.031759597, 0.025170002, -0.090425596, 0.048223794, 0.202898, -0.1456964, -0.062476397, 0.15210919, 0.15843621, 0.10579141, 0.59602606, -0.039108, 0.1184654, -0.04655268, -0.0051670047, 0.32464108, -0.191902, 0.0020141988, 0.0587912, -0.11395688, 0.18360801, -0.114694, -0.1219092, ...]"
32,49,ablaze,ablaze We always try to bring the heavy metal RT,0,ablaze we always try to bring the heavy metal RT,"[0.045223314, 0.1333939, -0.050343323, 0.08015067, 0.058838334, -0.0554654, -0.018545, 0.08634857, -0.051001832, 1.5571651, -0.10734125, -0.10247001, 0.13837694, 0.029538453, -0.079866245, -0.13390766, 0.044125587, 1.0139709, -0.1453875, 0.029887335, 0.09791225, -0.0023505837, -0.044185847, -0.060278002, -0.06502458, 0.016685084, -0.10645178, -0.074082986, 0.099368095, -0.07184333, 0.007993648, 0.0489028, -0.105595164, 0.121057004, 0.11047691, -0.023924187, 0.1304792, 0.09394637, 0.026132086, -0.04840246, -0.032212753, 0.023670083, -0.023923248, -0.19602084, 0.076127246, -0.04462072, 0.029789248, -0.10485867, -0.105620354, 0.07480307, 0.021529667, 0.101172, 0.09638619, 0.11223215, 0.029637339, 0.018065736, -0.16108476, -0.026385665, -0.051710915, -0.055401, -0.13327457, -0.04083883, -0.10042878, 0.14891459, 0.06408292, -0.06431223, 0.047274664, -0.042019755, 0.05255917, 0.059616495, -0.022005146, 0.0026573853, 0.110539995, 0.0010725869, 0.20260318, 0.01784075, 0.04255783, 0.009275745, -0.040025167, 0.0037872486, 0.018847583, 0.033973992, -0.23186707, -0.04028337, 0.10116502, -0.115347445, 0.037151914, 0.122359686, 0.1257649, 0.035692245, -0.072939165, -0.09851825, 0.08350283, 0.1036765, 0.012105833, 0.085228, 0.04201318, 0.017549252, -0.031674832, 0.004940662, ...]"
33,50,ablaze,ablaze AFRICANBAZE Breaking news Nigeria flag set ablaze in Aba,1,ablaze AFRICANBAZE break news Nigeria flag set ablaze in Aba,"[0.013710539, 0.14963509, 0.1158237, -0.021470385, -0.015969686, -0.08640769, -0.017800616, 0.11728731, -0.08619977, 1.017307, -0.15669984, -0.17209195, 0.0132249985, -0.039106157, -0.07251846, 0.004347696, 0.16670431, 0.5195946, 0.13820623, -0.13763155, -0.04797069, 0.014259151, 0.14222346, -0.041423623, 0.08688776, -0.04426492, -0.09938362, 0.13981031, 0.12833564, 0.0698008, 0.02618777, 0.0046466915, -0.07892693, 0.17477046, 0.059629228, 0.03305077, 0.022554385, 0.16193262, -0.033037618, -0.02115508, 0.10090612, 0.0062303073, 0.080808386, -0.07174353, -0.011538084, -0.10158007, 0.098384224, -0.04438454, -0.171915, -0.0044546905, -0.09982138, -0.027612928, -0.039756425, 0.029415388, -0.057097398, 0.009407693, 0.037795153, 0.053968847, 0.1611676, -0.06302346, -0.11900708, 0.120851554, -0.027273927, -0.030337691, 0.019895535, 0.014181694, 0.014484333, 0.0007102306, 0.12199794, 0.06954709, -0.03853, 0.010639996, 0.031774845, -0.08349548, 0.12122769, -0.0054409276, 0.029222231, -0.02278946, 0.051309228, 0.05586761, -0.05128553, 0.19871166, -0.034725077, 0.106526166, 0.12809578, 0.09827346, 0.17790054, 0.

In [12]:
nlp = spacy.load('en_core_web_lg')
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector for text in df_train.lemma_text])
    
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, df_train.target,
                                                    test_size=0.1, random_state=1)

In [14]:

# Set dual=False to speed up training, and it's not needed
svc = LinearSVC(random_state=1, dual=False)
svc.fit(X_train, y_train)
print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 76.984%


Thats.. A LOT better than  40+% we got from text classifier
Lets try other models to boost it up

In [24]:
# Lets try other classifiers
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 8, 1), random_state=1 , activation = 'relu')
mlp.fit(X_train, y_train)
print(f"Accuracy: {mlp.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 79.630%


Some improvement, lets try out some other ones

In [25]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_train)
print(f"Accuracy: {rfc.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 76.058%


In [28]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train, y_train)
print(f"Accuracy: {neigh.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 75.397%


### Alright, it seems like a neural network gives the best results, so lets hypertune it and continue


In [36]:
from sklearn.model_selection import RandomizedSearchCV
parameters = {'solver': ['sgd', 'adam'], 'activation':['identity','logistic','tanh','relu'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':[
        (10, 10), (20, 10), (30, 10),
        (40, 10), (90, 10), (90, 30, 10) ,(50, 8,1) , (100, 50, 1) , (25, 5 ,1 )  # <-- added more neurons or layers
    ], 'random_state':[0,1,2,3,4,5,6,7,8,9], 'learning_rate_init':[ 0.001, 0.0001, 0.00001] }


mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 8, 1), random_state=1 , activation = 'relu')
rscv = RandomizedSearchCV(mlp, parameters, n_iter=500, scoring='accuracy', n_jobs=-1, random_state=1)
search = rscv.fit(X_train,y_train)
print('best score:' ,search.best_score_ , 'params:', search.best_params_)

best score: 0.8119487728866381 params: {'solver': 'adam', 'random_state': 0, 'max_iter': 1900, 'learning_rate_init': 1e-05, 'hidden_layer_sizes': (90, 10), 'alpha': 1e-09, 'activation': 'relu'}


lets figure out  better hidden layer sizes

In [37]:
parameters = {'solver': ['adam'], 'activation':['relu'], 'max_iter': [1900 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':[
        (90, 15), (90, 5),
        (100, 10), (90, 10), (80, 10)   # <-- added more neurons or layers
    ], 'random_state':[0], 'learning_rate_init':[0.00001] }


mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 8, 1), random_state=1 , activation = 'relu')
rscv = RandomizedSearchCV(mlp, parameters, n_iter=500, scoring='accuracy', n_jobs=-1, random_state=1)
search = rscv.fit(X_train,y_train)
print('best score:' ,search.best_score_ , 'params:', search.best_params_)

best score: 0.812390273990391 params: {'solver': 'adam', 'random_state': 0, 'max_iter': 1900, 'learning_rate_init': 1e-05, 'hidden_layer_sizes': (90, 10), 'alpha': 0.001, 'activation': 'relu'}


In [39]:
mlp = MLPClassifier(solver='adam', learning_rate_init=1e-5, hidden_layer_sizes=(90,10), random_state= 0  , activation = 'relu',max_iter = 1900,alpha = 0.001)
mlp.fit(X_train, y_train)
print(f"Accuracy: {mlp.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 80.026%


looks like this is the best we can do with this so lets retrain with the whole model\
First we must prep our test data


In [46]:
df_test = pd.read_csv('test.csv')

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


Looks like we need to tweak our clean function to not remove nan values just for our test case

In [52]:
def clean_test_data(df_train):
    # remove locations
    df_train.drop(columns = ['location'],inplace=True)
    # fixing keywords from spaces
    df_train.keyword.fillna('',inplace=True)
    df_train.keyword = df_train.keyword.str.replace('%20',' ')
    # remove links and twitter @ tags
    df_train['text'] = df_train['text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
    df_train['text'] = [ re.sub( "@[^\s]+", "",x) for x in df_train['text'] ]
    # removing the # from texts
    df_train['text'] = df_train['text'].replace('#','')
    # Remove all punctuation
    df_train['text'] = df_train['text'].str.replace('[^\w\s]',' ')
    # add keyword to text
    df_train['text'] = df_train['keyword']+' '+ df_train['text']
    # lemmatize text
    nlp = spacy.load('en_core_web_md')
    lemma_text = []
    vector_text = []
    for text in df_train['text']:
        lemmatized_sentence = []
        doc = nlp(text)
        vector_text.append(doc.vector)
        for word in doc:
            lemmatized_sentence.append(word.lemma_)
        lemma_text.append(' '.join(lemmatized_sentence))
    df_train['lemma_text'] = lemma_text
    df_train['vector_text'] = vector_text
    

In [53]:
df_test = pd.read_csv('test.csv')
clean_test_data(df_test)

In [54]:
with nlp.disable_pipes():
    doc_test_vectors = np.array([nlp(text).vector for text in df_test.lemma_text])
    

Nice! our clean data function works

In [55]:
df_test.head()

,id,keyword,text,lemma_text,vector_text
0,0,,Just happened a terrible car crash,just happen a terrible car crash,"[-0.043289717, 0.26410273, -0.12693915, 0.13961129, 0.112736866, -0.13879843, 0.003558855, -0.19388571, -0.009975433, 2.1180143, -0.150581, -0.032039005, -0.014073713, 0.1756867, -0.46892142, -0.17533, -0.1389549, 0.79514647, 0.15214892, -0.15601686, -0.046058428, -0.17108844, 0.08387129, 0.028301997, 0.111256815, -0.026682856, -0.10912657, -0.10375301, 0.12266958, -0.07115484, 0.0068664295, -0.047060005, -0.074686, 0.19624087, 0.023135139, 0.03899128, 0.09167714, 0.10774473, -0.17946456, -0.010435998, 0.2712207, -0.016452143, 0.032477286, 0.122912146, 0.01276, 0.089147285, -0.2361467, -0.13553856, 0.038661003, -0.002578035, -0.031448144, 0.028147135, 0.101424575, 0.23548485, -0.12967357, 0.089275576, -0.019040141, -0.121329986, 0.20846856, -0.090907566, -0.06568063, -0.011910859, -0.051908426, 0.17539756, 0.11087315, -0.018365, 0.049989283, -0.06888928, 0.14956157, 0.2927057, 0.14276, 0.0045092874, 0.1568303, 0.16664828, 0.030409146, -0.106883354, 0.07174738, -0.18715987, -0.25888997, 0.2838443, 0.025382997, 0.095633574, 0.031191716, -0.022539238, -0.032312714, -0.018395139, 0.16677305, -0.27187872, -0.015319995, -0.089565, -0.18356471, 0.11526799, -0.19143489, -0.19904757, -0.010814284, 0.021742, 0.011040285, 0.115857854, 0.30231017, -0.03577843, ...]"
1,2,,Heard about earthquake is different cities stay safe everyone,hear about earthquake be different city stay safe everyone,"[0.017195191, 0.16088593, -0.19665648, -0.16031884, 0.07154349, -0.02444452, -0.040693168, -0.002709996, -0.0052762516, 1.8985084, -0.18281771, -0.07040492, -0.009747337, 0.046259835, -0.19452493, -0.08745184, -0.06155249, 0.75793415, -0.11244466, -0.10146925, 0.025320834, 0.033015583, 0.013431754, 0.023889942, 0.021379331, -0.020904293, -0.065634005, -0.03447062, 0.021395003, -0.13393499, 0.027691165, 0.0075789145, -0.008076832, -0.059751917, 0.0828395, 0.0066502406, 0.055584747, 0.06824492, -0.09942954, 0.040854625, -0.044708665, 0.023540331, -0.025846913, -0.011580084, 0.023103341, 0.05941258, -0.10884124, -0.10017568, -0.087092735, -0.19545817, -0.16438751, 0.083676584, -0.16831392, 0.005947503, 0.039432246, 0.098915316, 0.02432158, -0.044304613, 0.005236581, 0.14163482, 0.12321218, -0.1315089, -0.08092758, 0.11746333, 0.08942767, -0.08710692, -0.038330745, 0.10561309, -0.059010327, 0.17869167, -0.002543499, -0.052397165, 0.2927133, -0.059364587, 0.11506384, -0.0006590001, 0.11996784, -0.07575899, 0.052331995, 0.11761141, 0.18059099, 0.066438, -0.07590833, -0.040032998, 0.06721633, -0.14217083, -0.21622099, -0.16939764, 0.16191667, -0.044811413, -0.020456243, 0.014671435, -0.2429751, -0.043129165, 0.084597506, -0.03259254, -0.018928418, -0.034133498, 0.069916345, 0.04839717, ...]"
2,3,,there is a forest fire at spot pond geese are fleeing across the street I cannot save them all,there be a forest fire at spot pond geese be flee across the street I can not save they all,"[0.0026686348, 0.048581272, -0.17552017, -0.09000605, 0.094583504, 0.016473267, -0.021325609, -0.015770301, -0.13366878, 2.037452, -0.16842875, -0.039381344, -0.042325728, -0.14814202, -0.21142922, 0.023694223, -0.022793712, 0.99478084, -0.10071031, -0.061273534, -0.04664843, 0.00583326, -0.082282886, 0.04399414, 0.0072882264, 0.007162955, -0.11299938, -0.082375854, 0.017044043, -0.057016253, -0.053553473, 0.040617134, -0.082267925, 0.01913713, -0.018062174, -0.054832168, 0.038308047, -0.0131972125, -0.052933704, -0.08010658, -0.046223722, 0.11737744, -0.012798172, -0.045549218, 0.10582729, 0.07511165, -0.14597578, -0.05428021, -0.07933455, 0.050585523, -0.06696434, 0.060507745, -0.08408888, 0.088206075, 0.028901564, -0.03203939, -0.069300346, -0.08068513, 0.0025753903, 0.096970364, 0.029694824, -0.052423526, -0.053775694, 0.15258154, 0.016372692, 0.00018952269, -0.039335914, 0.029837662, 0.09896519, 0.011919611, 0.0062721437, 0.02643126, 0.17152144, 0.05274186, 0.101055

In [56]:
mlp = MLPClassifier(solver='adam', learning_rate_init=1e-5, hidden_layer_sizes=(90,10), random_state= 0  , activation = 'relu',max_iter = 1900,alpha = 0.001)
mlp.fit(doc_vectors, df_train.target)
pred_test = mlp.predict(doc_test_vectors)
pred_test[:5]

array([0, 1, 1, 1, 1], dtype=int64)

In [62]:
submit = pd.DataFrame( [df_test['id'], pred_test] , index= ['id','target'] ).T
submit.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [63]:
submit.to_csv('submission.csv',index = False)

# Got a 0.80049 on kaggle! #352 on the leaderboard

### Possible improvements
- further text editing, removing nonsense words/non-english words 
- other text fixing like weird symbols or spacing errors